In [100]:
#Import modules
import numpy as np
import pandas as pd
import random

In [101]:
real_decal = pd.read_excel("data\Condensed Spreadsheet for Grace and Bradley.xlsx")
real_decal.columns = ["Email","SID","Name","Availabilities","Driver1", "Driver2", "Spanish", "Comments"]
real_decal = real_decal[["Name", "Email", "Availabilities","Driver1","Driver2", "Spanish"]]
real_decal.head()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\grace\AppData\Local\Temp\ipykernel_25480\3059901807.py:1: SyntaxWarning: invalid escape sequence '\C'
  real_decal = pd.read_excel("data\Condensed Spreadsheet for Grace and Bradley.xlsx")


,Name,Email,Availabilities,Driver1,Driver2,Spanish
0,Owen Spomer,ospomer@berkeley.edu,"Tuesday 3:30-4:30pm, Tuesday 3:45-4:45pm*, Tue...",No,No,No
1,Terry Yesung Han,terryyesunghan@berkeley.edu,Friday 1:30-2:30pm,Yes,Yes,No
2,Maxwell Edison Stolarz,maxwellstolarz@berkeley.edu,"Tuesday 5:00-6:00pm*, Wednesday 1:00-2:00pm, T...",No,No,No
3,Aaron Presley Garcia,dumbeatz@berkeley.edu,"Friday 1:30-2:30pm, Friday 3:15-4:15pm",No,Yes,No
4,Puteri Sarah Bt. Megat Harris,sarahmegatharris@berkeley.edu,"Wednesday 12:00-1:00pm*, Thursday 2:15-3:15pm,...",No,No,No


In [102]:
accepted_decal = pd.read_excel("..\\Decal Roster Crosscheck\\Non-Waitlisted Decal Members.xlsx")
accepted_decal_emails = accepted_decal[['Email']]
decal = real_decal.merge(accepted_decal, on = "Email").iloc[:,:6]
decal['Availabilities'] = decal['Availabilities'].str.split(", ")

In [103]:
def get_sitetimes_list(decal):
    full_set = set()
    for lst in decal["Availabilities"]:
        if type(lst) == list:
            for item in lst:
                full_set.add(item.strip())
    return full_set

def ohe_availabilities(decal, sitetimes):
    availbilities = decal['Availabilities']
    new_df = pd.DataFrame()
    for time in sitetimes:
        ohe = []
        for person in availbilities:
            if type(person) != float:
                ohe.append(time in person)
        new_df[time] = ohe
    full_df = decal.join(new_df)
    return full_df

def merge_driving_status(decal):
    d1 = decal["Driver1"]
    d2 = decal["Driver2"]

    status = []
    for i in range(decal.shape[0]):
        status.append(d1[i] == "Yes" or d2[i] == "Yes")
    decal = decal.drop(columns = ["Driver1","Driver2"])
    decal["Driver"] = status
    return decal

def fix_spanish(decal):
    status = []
    spanish = decal["Spanish"]
    for i in range(decal.shape[0]):
        status.append(spanish[i] == "Yes")
    decal["Spanish"] = status

    return decal

In [104]:
sitetimes = list(get_sitetimes_list(decal))
decal = ohe_availabilities(decal, sitetimes)
decal = merge_driving_status(decal)
decal = fix_spanish(decal)

In [108]:
decal.to_csv("Updated Decal.csv")